Генетический метод

In [ ]:
import tkinter as tk
from tkinter import ttk
import random


PEAK_HOURS = [(7, 9), (17, 19)]
SHIFT_DURATION = 9
DRIVER_TYPES = ["A", "B"]
POPULATION_SIZE = 20
GENERATIONS = 100
MUTATION_RATE = 0.1
BREAK_DURATION_BIG = 1
BREAK_DURATION_SMALL = 0.25
BREAK_INTERVAL = 3


drivers = [
    {"name": "Водитель 1", "start": 6, "type": "A"},
    {"name": "Водитель 2", "start": 7, "type": "A"},
    {"name": "Водитель 3", "start": 8, "type": "B"},
    {"name": "Водитель 4", "start": 10, "type": "A"},
    {"name": "Водитель 5", "start": 11, "type": "B"},
    {"name": "Водитель 6", "start": 13, "type": "B"},
    {"name": "Водитель 7", "start": 16, "type": "A"},
    {"name": "Водитель 8", "start": 18, "type": "B"},
]


def is_peak(hour):
    for start, end in PEAK_HOURS:
        if start <= hour < end:
            return True
    return False


def random_schedule(driver):
    shift_start = driver["start"]
    shift_hours = [(shift_start + i) % 24 for i in range(SHIFT_DURATION)]
    breaks = random.sample(shift_hours, k=random.randint(1, 2))
    return {
        "hours": shift_hours,
        "breaks": sorted(breaks),
    }


def fitness(schedule):
    score = 0
    for driver, data in schedule.items():
        breaks = data["breaks"]
        for hour in breaks:
            if is_peak(hour):
                score -= 1
            else:
                score += 1
    return score


def initialize_population():
    return [
        {driver["name"]: random_schedule(driver) for driver in drivers}
        for _ in range(POPULATION_SIZE)
    ]


def crossover(parent1, parent2):
    child = {}
    for driver in drivers:
        name = driver["name"]
        if random.random() < 0.5:
            child[name] = parent1[name]
        else:
            child[name] = parent2[name]
    return child


def mutate(schedule):
    for driver in drivers:
        if random.random() < MUTATION_RATE:
            name = driver["name"]
            schedule[name] = random_schedule(driver)


def genetic_algorithm():
    population = initialize_population()
    best_schedule = None
    best_fitness = float("-inf")

    for generation in range(GENERATIONS):
        population = sorted(population, key=fitness, reverse=True)

        if fitness(population[0]) > best_fitness:
            best_schedule = population[0]
            best_fitness = fitness(population[0])

        next_generation = population[:POPULATION_SIZE // 2]

        while len(next_generation) < POPULATION_SIZE:
            parent1, parent2 = random.sample(population[:POPULATION_SIZE // 2], 2)
            child = crossover(parent1, parent2)
            mutate(child)
            next_generation.append(child)

        population = next_generation

    return best_schedule



def create_gui(schedule):
    root = tk.Tk()
    root.title("Расписание водителей автобусов")
    root.geometry("1800x700")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(xscrollcommand=scrollbar.set)

    canvas.pack(side="top", fill="both", expand=True)
    scrollbar.pack(side="bottom", fill="x")

    for idx, (driver, data) in enumerate(schedule.items()):
        frame = ttk.LabelFrame(scrollable_frame, text=driver, padding=10)
        frame.grid(row=0, column=idx, padx=10, pady=5, sticky="n")

        shift_hours = data["hours"]
        breaks = data["breaks"]

        for hour in shift_hours:
            if hour in breaks:
                label = ttk.Label(frame, text=f"{hour:02d}:00 - Перерыв", anchor="w", padding=5)
            elif is_peak(hour):
                label = ttk.Label(frame, text=f"{hour:02d}:00 - Поездка (час пик)", anchor="w", padding=5)
            else:
                label = ttk.Label(frame, text=f"{hour:02d}:00 - Поездка", anchor="w", padding=5)
            label.pack(fill="x")

    root.mainloop()


if __name__ == "__main__":
    best_schedule = genetic_algorithm()
    create_gui(best_schedule)
